# Task 9 - Diffie-Hellman-Schlüsselaustausch
Krypto Lab

Felix Kleinsteuber, 185 709

In [1]:
import numpy as np
import random

## 1. Finde Primzahl p und Erzeuger g
Diese werden zum Schlüsselaustausch benötigt und sind global bekannt. Praktisch nehmen wir ein standardisiertes $g = 2$, welches nicht zwingend ein Generator ist.

In [2]:
# aus Task 7 (RSA Keygen)
def miller_rabin_test(n):
    """Ist n prim? Gibt mit p < 1/4 True zurück, falls n zusammengesetzt ist (False Positive)
    und sicher True, falls n eine Primzahl ist."""
    assert n > 2
    # Bestimme ungerades m mit n - 1 = 2^k * m
    m = n - 1
    k = 0
    while m & 0b1 == 0:
        m >>= 1
        k += 1
    
    # Wähle zufälliges 2 <= a < n
    a = random.randrange(2, n)

    # b = a^m mod n
    b = quad_and_mult(a, m, n)

    if b == 1:
        return True
    for i in range(k):
        if b == n - 1:
            return True
        else:
            b = (b * b) % n
    return False

# aus Task 7 (RSA Keygen)
def is_prime(n):
    """Gibt True zurück, wenn mit sehr hoher Wahrscheinlichkeit (0.999999) prim ist. """
    for i in range(10):
        if miller_rabin_test(n) == False:
            return False
    # mit p > 1 - (1/4)**10 = 0.999999... ist n prim
    return True

In [3]:
# Abgewandelt von look_for_prime (RSA Keygen)
def look_for_q(z):
    # Sucht eine Primzahl q > 30 * z, sodass p = 2q + 1 prim
    for i in [1, 7, 11, 13, 17, 19, 23, 29]:
        q = 30 * z + i
        if is_prime(q) and is_prime(2 * q + 1):
            return q
    # Keine Primzahl gefunden, erhöhe z
    return look_for_q(z + 1)

def find_pg(min_z=2**128, max_z=2**256):
    q = look_for_q(random.randrange(min_z, max_z))
    p = 2 * q + 1
    # Als "Erzeuger" immer 2 verwenden
    # (2 ist nicht zwingend immer Erzeuger aber erzeugt einen großen Teil von Z_p*)
    return p, 2

## 2. Diffie-Hellman

In [4]:
# aus Task 6
def quad_and_mult(x, m, n):
    """Berechnet effizient x^m mod n."""
    y = 1
    while m != 0:
        if m & 0x1 != 0:
            # falls bit=1: Multipliziere y mit x
            y = (y * x) % n
        # für jedes Bit wird x quadriert
        x = (x * x) % n
        # schiebe zum nächsten Bit
        m >>= 1
    return y

Einmal im Schnelldurchlauf:

In [5]:
p, g = find_pg()
# Alice generiert Zufallszahl a
a = random.randrange(2, p)
# Bob generiert Zufallszahl b
b = random.randrange(2, p)
# Beide berechnen die Potenz des Generators und schicken das Ergebnis an den anderen
A = quad_and_mult(g, a, p)
B = quad_and_mult(g, b, p)
# Beide berechnen jetzt die Potenz des Generators mit dem Ergebnis des anderen
S_A = quad_and_mult(B, a, p)
S_B = quad_and_mult(A, b, p)
# beide erhalten S_A == S_B == (g ** a) ** b == g ** (a * b)
assert S_A == S_B

Getrennt für Alice und Bob:

In [6]:
class Person:
    def __init__(self, p, g):
        self.p = p
        self.g = g
    
    def shared_secret(self):
        self.secret = random.randrange(2, self.p)
        return quad_and_mult(self.g, self.secret, self.p)
    
    def get_key(self, other_shared_secret):
        return quad_and_mult(other_shared_secret, self.secret, self.p)

In [7]:
p, g = find_pg()
# Beide kennen p, g
Alice = Person(p, g)
Bob = Person(p, g)

A = Alice.shared_secret()
print(f"Alice teilt {A}")
B = Bob.shared_secret()
print(f"Bob teilt {B}")

S_A = Alice.get_key(B)
print(f"Alice kennt den Schlüssel {S_A}")
S_B = Bob.get_key(A)
print(f"Bob kennt den Schlüssel {S_B}")
assert S_A == S_B

Alice teilt 2751548103124226825963230020500857213083444926882394710700976065328801444973007
Bob teilt 958628531017096319275464681651782661190113558148939485523064539590776870251755
Alice kennt den Schlüssel 2030821126039866076617933281259704826303124873203752463921450283613029828387933
Bob kennt den Schlüssel 2030821126039866076617933281259704826303124873203752463921450283613029828387933


Eve könnte sich nun als Alice oder Bob ausgeben (Man-in-the-Middle-Angriff). Das kann durch die Signatur der Nachrichten verhindert werden.